In [1]:
DRIVE_PATH = ''
mode = 'roberta' 
task_name = 'trip'
debug = False
config_batch_size = 1
config_lr = 1e-6 
config_epochs = 15
import os
import json
import sys
import torch
import random
import numpy as np
import spacy
model_name = 'roberta-large'
from transformers import RobertaTokenizerFast
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-large')
from transformers import RobertaConfig,RobertaModel,AdamW
config_class = RobertaConfig
emb_class = RobertaModel
from www.utils import print_dict
import json
from collections import Counter


data_file = os.path.join('../Source_task/trip.json')
with open(data_file, 'r') as f:
    cloze_dataset_2s, order_dataset_2s = json.load(f)  

for p in cloze_dataset_2s:
    label_dist = Counter([ex['label'] for ex in cloze_dataset_2s[p]])
    print('Cloze label distribution (%s):' % p)
    print(label_dist.most_common())

Cloze label distribution (train):
[(1, 400), (0, 399)]
Cloze label distribution (dev):
[(0, 161), (1, 161)]
Cloze label distribution (test):
[(1, 176), (0, 175)]


#### Load trip dataset

In [1]:
from www.dataset.prepro import get_tiered_data, balance_labels
from www.dataset.adaptation_joint import add_bert_features_tiered_modify,get_tensor_dataset_tiered_modify,filter_dataset,getMaxStoryLength,checkdistribution
from collections import Counter
tiered_dataset = cloze_dataset_2s

# Debug the code on a small amount of data
debug = True
if debug:
    for k in tiered_dataset:
        tiered_dataset[k] = tiered_dataset[k][:10]

maxStoryLength=168       
tiered_dataset = get_tiered_data(tiered_dataset)
tiered_dataset = add_bert_features_tiered_modify(tiered_dataset, tokenizer,maxStoryLength, add_segment_ids=True)


tiered_tensor_dataset = {}
max_story_length = max([len(ex['stories'][0]['sentences']) for p in tiered_dataset for ex in tiered_dataset[p]])
for p in tiered_dataset:
    tiered_tensor_dataset[p] = get_tensor_dataset_tiered_modify(tiered_dataset[p], max_story_length, maxStoryLength,add_segment_ids=True)
    
    
from www.dataset.ann import att_to_idx, att_to_num_classes, att_types

subtask = 'cloze'
batch_sizes = [config_batch_size]
learning_rates = [config_lr]
epochs = config_epochs
eval_batch_size = 16
generate_learning_curve = True # Generate data for training curve figure in TRIP paper

num_state_labels = {}
for att in att_to_idx:
    if att_types[att] == 'default':
        num_state_labels[att_to_idx[att]] = 3
    else:
        num_state_labels[att_to_idx[att]] = att_to_num_classes[att] # Location attributes fall into this since they don't have well-define pre- and post-condition yet

ablation = ['attributes', 'states-logits'] # This is the default mode presented in the paper    

#### Parameter setting
To choose **sentence-centric** set **conflict_pred** to True

To choose **entity-centric** set **conflict_pred** to False and **story_label** to False

To choose **story-centric** set **conflict_pred** to False and **story_label** to True

In [3]:
story_label=True
conflict_pred=True



DRIVE_PATH=''
story_loss=None 
story_back=None
loss_percent=None
if conflict_pred:
    story_loss=False
    story_back=False
    loss_percent = [0.4,0.4,0.2,0]
    DRIVE_PATH="Single_Model_sentence_centric"
else:
    story_loss=True 
    story_back=True
    loss_percent = [0.4,0.4,0.1,0.1]
    if story_label:
        DRIVE_PATH="Single_Model_story_centric"
    else:
        DRIVE_PATH="Single_Model_entity_centric"

#### Load model

In [2]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, f1_score
from www.utils import print_dict, get_model_dir
# Modify
from Method_Single import compute_metrics,update_result,trainModel,trainModel_config,evalModel,evaluation,verifiable_reasoning,save_results
from Model_single import RobertaProceduralTSLM
from www.dataset.ann import att_to_num_classes
import shutil
import pandas as pd
from tqdm import tqdm


seed_val = 24 # Save random seed for reproducibility
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

train_sampler = RandomSampler(tiered_tensor_dataset['train'])
train_dataloader = DataLoader(tiered_tensor_dataset['train'], sampler=train_sampler, batch_size=1)
dev_sampler = SequentialSampler(tiered_tensor_dataset['dev'])
dev_dataloader = DataLoader(tiered_tensor_dataset['dev'], sampler=dev_sampler, batch_size=1)

config = config_class.from_pretrained(model_name)  
device="cuda:0"
num_attributes=len(num_state_labels)
max_story_length = max([len(ex['stories'][0]['sentences']) for p in tiered_dataset for ex in tiered_dataset[p]])



tslm= RobertaProceduralTSLM.from_pretrained('tli8hf/unqover-roberta-large-squad', return_dict=True,num_attributes=num_attributes,labels_per_att=num_state_labels,story_loss=story_loss,story_back=story_back).to(device)
tslm_optimizer = AdamW(tslm.parameters(), lr=config_lr)
# total_steps = len(train_dataloader) * epochs
# scheduler = get_linear_schedule_with_warmup(tslm_optimizer, num_warmup_steps=0, num_training_steps = total_steps)


#### Train model within source task`

In [3]:
print('\nTRAINING MODEL')
best_accuracy=0
best_verifiability=0
best_accuracy_dir=""
best_verifiability_dir=""
grad_accmu=2
for epoch_number in range(epochs):
    print("Start Training in epoch #{}".format(epoch_number))
    train_lc_data = []
    tslm.train()
    for layer in tslm.precondition_classifiers:
        layer.train()
    for layer in tslm.effect_classifiers:
        layer.train()    
    for batch in tqdm(train_dataloader):
        final_out=trainModel_config(batch,max_story_length,maxStoryLength,num_attributes,device\
                             ,tslm,tslm_optimizer,story_label=story_label,grad_accmu=grad_accmu,loss_percent=loss_percent)
        train_lc_data.append({'loss_preconditions':float(final_out['loss_preconditions'].detach().cpu().numpy()),
                              'loss_effects': float(final_out['loss_effects'].detach().cpu().numpy()),
                              'loss_conflicts': float(final_out['loss_conflicts'].detach().cpu().numpy()),
                              'loss_stories': float(final_out['loss_story'].detach().cpu().numpy()),
                              'loss_total': float(final_out['total_loss'].detach().cpu().numpy())})
        
    print("Start Evaluation in epoch #{}".format(epoch_number))
    tslm.eval()
    for layer in tslm.precondition_classifiers:
        layer.eval()
    for layer in tslm.effect_classifiers:
        layer.eval()    
    metr_prec,metr_eff,metr_conflicts,metr_stories,verifiability,consistency, explanations=evaluation(max_story_length,maxStoryLength,num_attributes,tslm,tslm_optimizer,dev_dataloader,device,story_label=story_label,conflict_pred=conflict_pred)
    accuracy=metr_stories['accuracy']
    metr_stories['consistency']=consistency
    metr_stories['verifiability']=verifiability
    
    # save metrics
    task_name="trip_%s_dev"
    output_model_path=os.path.join(DRIVE_PATH,str(epoch_number))
    if not os.path.exists(output_model_path):
        os.makedirs(output_model_path)
    save_results(metr_prec, output_model_path, task_name % 'preconditions')
    save_results(metr_eff, output_model_path, task_name % 'effects')
    save_results(metr_conflicts, output_model_path, task_name % 'conflicts')
    save_results(metr_stories, output_model_path, task_name % 'stories')
    save_results(explanations, output_model_path, task_name % 'explanations')   
    train_lc_data = pd.DataFrame(train_lc_data)
    train_lc_data.to_csv(os.path.join(output_model_path, 'learning_curve_data_train.csv'), index=False)

    
    model_dir=os.path.join(output_model_path, 'tslm.pth')
    torch.save(tslm, model_dir)
    
    if best_accuracy < accuracy:
        best_accuracy=accuracy
        best_accuracy_dir=model_dir
    if best_verifiability < verifiability:
        best_verifiability=verifiability
        best_verifiability_dir=model_dir
    print("accuracy {}".format(accuracy))
    print("consistency {}".format(consistency))
    print("verifiability {}".format(verifiability))
print("Traing Process Finish")
print("Achieve best accuracy {} at {}".format(best_accuracy,best_accuracy_dir))
print("Achieve best verifiability {} at {}".format(best_verifiability,best_verifiability_dir))


#### Evaluation

In [9]:
for p in tiered_dataset:
    if p != 'test':
        continue

    p_dataset = tiered_dataset[p]
    p_tensor_dataset = tiered_tensor_dataset[p]
    p_sampler = SequentialSampler(p_tensor_dataset)
    p_dataloader = DataLoader(p_tensor_dataset,
                              sampler=p_sampler,
                              batch_size=1)
    task_name = 'trip' + '_%s_' + p

In [4]:
Dir_file=DRIVE_PATH
for i in range(15):
    file_number=str(i)
    output_model_path=os.path.join(Dir_file, file_number)
    tslm = torch.load(os.path.join(output_model_path, 'tslm.pth'))
    print("Start Evaluation in epoch #{}".format(i))
    tslm.eval()
    metr_prec,metr_eff,metr_conflicts,metr_stories,verifiability,consistency, explanations=evaluation(max_story_length,maxStoryLength,num_attributes,tslm,tslm_optimizer,p_dataloader,device,story_label=story_label,conflict_pred=conflict_pred)
    accuracy=metr_stories['accuracy']
    metr_stories['consistency']=consistency
    metr_stories['verifiability']=verifiability
    print("accuracy {}".format(accuracy))
    print("consistency {}".format(consistency))
    print("verifiability {}".format(verifiability))
    save_results(metr_prec, output_model_path, task_name % 'preconditions')
    save_results(metr_eff, output_model_path, task_name % 'effects')
    save_results(metr_conflicts, output_model_path, task_name % 'conflicts')
    save_results(metr_stories, output_model_path, task_name % 'stories')
    save_results(explanations, output_model_path, task_name % 'explanations') 